#### from list to rows; Patient_id, Date - changed format, new col - Age; removed redundunt cols

In [1]:
!pip install pandas

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_json('Data_ICD_E11_T2DM_1.json')

In [6]:
df.shape


(6072, 117)

In [30]:
## testing code!!
len([i for i in df[df.index == 'GACAUЧ-']['Glu (vein)'][0] if i != 0])

10

In [7]:
df_n = pd.DataFrame({col: df[col].apply(pd.Series).stack().reset_index(drop=True) for col in df.columns})

In [8]:
# Patient id col 
df_n['Patient_ID'] = pd.Series([i[:7] for i in df_n.EventCode])

In [9]:
# new Date format, DOB to date-time format
df_n['Date'] = pd.to_datetime(df_n.Date.astype('str'), format='%Y%m%d', errors='ignore')
df_n['Birthday'] = pd.to_datetime(df_n.Birthday)

In [10]:
# Age of patients at event time
df_n['Age'] = pd.to_datetime(df_n.Date).dt.year - pd.to_datetime(df_n.Birthday).dt.year

In [7]:
df_n.shape

(1550560, 117)

In [10]:
#export to csv
df_n.to_csv('Dataset_by_rows_w_Age_DateOfEvent_PatientID.csv')

In [17]:
# Vlad's feature selection
the_most_interesting_element_count = 1000
second_priority_element_count = 50
features_with_counts = {}
remove = []
for col in df_n.columns:
    features_with_counts[col] = np.count_nonzero(df_n[col] != 0)
sorted_features = {k: v for k, v in sorted(features_with_counts.items(), key=lambda item: item[1], reverse=True)}
for key in sorted_features:
    if sorted_features[key] <= 95:
        remove.append(key)

In [11]:
# redundunt features deletion
df_clear = df_n.drop(columns = remove + ['E0061epizod', 'B0050.1', 
                                         'all_gipoglemic_drugs', 'beta_block', 'Birthday']).set_index('EventCode')

NameError: name 'remove' is not defined

In [ ]:
df_clear.head()

In [27]:
#export to csv
df_clear.to_csv('Preprocessed.csv')

In [12]:
for i in df_n.columns

Index(['EventCode', 'Date', 'Birthday', 'Sex', 'Smoke_status', 'HbA1c',
       'Protein (urine) amount.', 'Microalbumin urine',
       'Microalbin (day urine)', 'Metanephrines urine',
       ...
       'beta_block', 'IM1', 'CHD1', 'diabetic_retinopathy1',
       'diabetic_angiopathy_neph', 'diabetic_nephropathy1', 'neuropathy1',
       'stroke1', 'Patient_ID', 'Age'],
      dtype='object', length=119)